2023年度前期

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/banner.png?raw=true" height="120px">

# 解析実習 前半


　今回の実習では、ナノポアシーケンシングにより得られたゲノム情報を使って、遺伝子系統樹を作り、植物種間・種内の系統関係を調べていきます。


## 解析実習の流れ
　このテキストでは「解析実習1」をおこないます。

- 解析実習1（シーケンシングリード処理、変異検出、配列決定）
  1. シーケンシングリードとそのクオリティを確認する
  1. リードのフィルタリングをおこなう
  1. リードを参照配列にアライメントする
  1. アライメント結果を確認する
  1. 実習サンプルの変異を検出する
  1. 実習サンプルの塩基配列を決定する
- 解析実習2（系統解析） ※[後半パートのテキスト](https://colab.research.google.com/github/CropEvol/lecture/blob/master/FFBCexpr_2023spr/FFBC2023spr_CropEvol_p2.ipynb)



## 事前準備: ソフトウェアをインストールする

　実習前に次のコードセルを実行してください。

In [ ]:
### 実習前にこのコードセルを実行してください
!wget -q -O Install_softwares_p1.sh https://raw.githubusercontent.com/CropEvol/lecture/master/FFBCexpr_2023spr/scripts/Install_softwares_p1.sh
!sh Install_softwares_p1.sh

## 事前準備: シーケンシングデータのダウンロード
　下記の実習サンプル名をどれかひとつ選び、そのサンプル名を`sample_name=`の右に記入して、コードセルを実行してください。シーケンシングデータ（`サンプル名.fastq`）をダウンロードできます。

コムギ近縁野生種:
- *Aegilops longissima* LON-2 （入力サンプル名: `LON_2`）
- *Aegilops bicornis* BIC-1 （`BIC_1`）
- *Aegilops bicornis BIC-2* （`BIC_2`）

オニドコロ *Dioscorea tokoro*:
- 東北・雄株 （`NM`）
- 東北・雌株 （`NF`）
- 近畿・雄株 （`CM`）
- 近畿・雌株 （`CF`）
- 九州・雄株 （`SM`）
- 九州・雌株 （`SF`）

　また、このあとの解析のための参照配列もダウンロードします。

参照配列のファイル名:
- コムギ近縁野生種: `Aegilops_ref.fasta`
- オニドコロ: `Dioscorea_ref.fasta`

In [ ]:
%%bash
## 実習サンプル
sample_name=SF
wget -q -O ${sample_name}.fastq https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fastq/${sample_name}.fastq?raw=true
ls ${sample_name}.fastq
## 参照配列
wget -q -O Aegilops_ref.fasta   https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/ref/Ae_tauschii_chloroplast_genome.fasta?raw=true
wget -q -O Dioscorea_ref.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/ref/D.tokoro.chloroplast_genome.v1.0.fasta?raw=true

## 実際のゲノム解析との違い

　実際のゲノム解析では、今回の勉強するような処理をGoogle Colab上でおこなうことはありません。UNIXやLinuxといったコンピュータにソフトウェアをインストールして、ゲノム解析をおこないます。その際、コマンドラインインターフェイス（WindowsではPowerShell、Macではターミナル）と呼ばれるアプリを使い、コンピュータに指示を出し、ゲノム解析をおこないます。

- [コマンドラインインターフェイス操作の動画](https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/movies/CUI_proc.md)

　今回の実習では、コマンドラインインターフェイスに入力する指示文（コマンド）をGoogle Colabのコードセルに書き込み、ゲノム解析をおこないます。以降のテキストでは、`%%bash`から始まるコードセルでの処理がコマンドラインインターフェイスでおこなう処理に相当します。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/cui.png?raw=true" height="250px">

## 1. シーケンシングリードとそのクオリティを確認する



### 1a. リードの確認
　まず最初に、どのようなデータが得られたか確認してみましょう。ナノポアのような次世代シーケンシングをおこなうと FASTQ形式と呼ばれるデータが得られます。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/nanopore_output.png?raw=true" height="200px">

　次のようなコードを書いて、実際にFASTQ形式のデータを見てみましょう。
```sh
## headコマンド（ファイルの先頭数行を出力する）
head -n 20 SAMPLE.fastq

## オプションの説明
# -n: 出力する行数
```

In [ ]:
%%bash
head -n 20 SAMPLE.fastq

### 1b. リード情報の確認（`NanoStat`）

　得られたシーケンシングリードの数や質を確認してみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/read_check.png?raw=true" height="250px">

　今回、`NanoStat` と呼ばれるソフトウェアを使って、リード情報の確認をおこないます。

```sh
## NanoStatの使い方
NanoStat --fastq SAMPLE.fastq

## オプションの説明
# --fastq: 入力ファイル（シーケンシングリードファイル）
```

　`SAMPLE.fastq`の箇所にシーケンシングリードのファイル名を入れて実行してください。

In [ ]:
%%bash
NanoStat --fastq SAMPLE.fastq

### 1c. リード情報の確認（`NanoPlot`）

　別のソフトウェア `NanoPlot` を使うと、より多くの情報を得られます。

```sh
## NanoPlotの使い方
NanoPlot --fastq SAMPLE.fastq -o OUTPUT_FOLDER

## オプションの説明
# --fastq: 入力ファイル（シーケンシングリードファイル）
# -o: 出力先フォルダ名
```

In [ ]:
%%bash
NanoPlot --fastq SAMPLE.fastq -o SAMPLE

## 2. リードのフィルタリングをおこなう



### 2a. リードフィルタリング

　シーケンシング後のリードには、質の低いリードが含まれています。そのようなリードを含んだままこのあとの解析を進めると、良い結果が得られないことがあります。質の低いリードを除いて（**フィルタリング**し）、質の良いリードのみを残しましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/read_control.png?raw=true" height="250px">

　ここでは、`NanoFilt` と呼ばれるソフトウェアを使用して、リードのフィルタリングをおこないます。

```sh
## NanoFiltの使い方
NanoFilt -q 8 SAMPLE.fastq > SAMPLE.filt.fastq

## オプション等の説明
# -q: 許容するリードの平均クオリティ
# >: 出力先ファイル
```

In [ ]:
%%bash
NanoFilt

### 2b. フィルタリング結果の確認

　ステップ1bで使用した `NanoFilt` で、フィルタリング後のリードの情報を確認してみてください。

In [ ]:
%%bash
NanoStat

　余裕がある方は、 `NanoPlot` でもフィルタリング後の結果を確認してみてください。

In [ ]:
%%bash
NanoPlot

## 3. リードを参照配列にアライメントする

　アライメントソフトウェア `bwa` を使って、フィルタリングしたリードを**参照配列（リファレンス配列）**にアライメントしましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/read_alignment.png?raw=true" height="130px">

### 3a. 参照配列のインデックス（`bwa index`）

　アライメントのためには、まず参照配列（リファレンス配列）のインデックスファイルと呼ばれるものを作成する必要があります。

```sh
## インデックスファイルの作成
## bwa indexの使い方
bwa index REF.fasta
```

参照配列のファイル名:
- コムギ近縁野生種: `Aegilops_ref.fasta`
- オニドコロ: `Dioscorea_ref.fasta`

In [ ]:
%%bash
bwa index

### 3b. リードのアライメント（`bwa mem`）

　参照配列のインデックスを作成後、リードを参照配列に**アライメント**（**マッピング**）します。

```sh
## 参照配列へのリードアライメント
## bwa memの使い方
bwa mem -x ont2d REF.fasta SAMPLE.filt.fastq > SAMPLE.sam

## オプション等の説明
# -x: リードの種類 （ont2dはナノポアシーケンシングリード）
# >: 出力先ファイル。SAM形式と呼ばれるデータが出力されるため、「.sam」の拡張子を付けるのがよい
```


In [ ]:
%%bash
bwa mem

### 3c. アライメントファイルの並べ替え・形式変換（`samtools sort`）

　リードアライメント後のファイルには、アライメント時の処理順にリードが格納されています。後ほどおこなうアライメント状況の確認や変異検出の際には、アライメントされたリードがゲノム上のポジション順に並べられている必要があります。その並べ替えをおこないましょう。

　また、コンピュータが処理しやすいように、SAM形式をBAM形式（SAMのバイナリ化形式）に変換する処理も同時におこないましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/read_sorting.png?raw=true" height="180px">

　`samtools sort` コマンドで以下ふたつのことを一括でおこなえます。
- アライメントリードの並べ替え
- SAM形式をBAM形式に変換する

```sh
## samtools sortの使い方
samtools sort -O bam SAMPLE.sam > SAMPLE.bam

## オプション等の説明
# -O: 出力形式 （bamはBAM形式での出力）
# >: 出力先ファイル。BAM形式で出力されるため「.bam」の拡張子を付けるのがよい
```

In [ ]:
%%bash
samtools sort

## 4. アライメント結果を確認する

　ここでは、IGVというツールを使って、リードのアライメント結果を確認します。

　IGVにデータを渡すためには、参照配列（例では`REF.fasta`）の `samtools` のインデックスファイル（`bwa` で作ったインデックスファイルとは別のもの）が必要です。まずは、そのインデックスファイルを作ります。


参照配列のファイル名:
- コムギ近縁野生種: `Aegilops_ref.fasta`
- オニドコロ: `Dioscorea_ref.fasta`

```sh
## samtools faidxの使い方
samtools faidx REF.fasta
```


In [ ]:
%%bash
samtools faidx

　次に、IGVでアライメント結果を確認しましょう。

In [ ]:
### これはPythonプログラムです ###

## 参照配列（FASTA形式ファイル）
ref = ""
## BAM形式ファイル（ステップ3cで準備したファイル）
sample = ""

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": ref, "indexPath": ref+".fai", "id": ref })
bam = BamTrack({ "name": sample, "path": sample, "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(bam)

　各サンプルのアライメント結果はこちらで見れます。
- [コムギ近縁野生種 4種](https://tinyurl.com/28rey4xs) *Aegilops longissima*, *Ae. sharonensis*, *Ae. bicornis*, *Ae. searsii*
- [オニドコロ](https://tinyurl.com/28ewmgyl) *Dioscorea tokoro*



## 5. 実習サンプルの変異を検出する

　リードアライメント結果を使って、変異を検出してみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/variant_calling.png?raw=true" height="190px">

　変異を検出するには、二つの`bcftools`コマンド `mpileup` と `call` を使います。

```sh
## bcftools mpileup / call　の使い方
bcftools mpileup -f REF.fasta SAMPLE.bam | bcftools call -O z -v > SAMPLE.vcf.gz

## mpileupオプションの説明
# -f: 参照配列
## callオプションの説明
# -O: 出力形式（zはvcf.gz形式での出力を指定）
# -v: 検出されたvariantのみを出力する
## パイプ(|)
# mpileupの出力を別のコマンドに渡す
```


In [ ]:
%%bash
bcftools mpileup | bcftools call

　変異検出の結果（vcf.gz形式ファイル）を確認してみましょう。

```
## vcf.gzの内容確認方法
zcat SAMPLE.vcf.gz
```

In [ ]:
%%bash
zcat

#### パイプ(|)についての補足

　パイプ（`|`）を使うと、前のコマンドの出力結果を、次のコマンドの入力値として渡すことができます。

　例えば、`zcat`による出力内容から、`grep`で「#」を含む行のみを検索し、その行のみを出力することも可能です。

In [ ]:
%%bash
zcat SAMPLE.vcf.gz | grep '#'

## 6. 実習サンプルの塩基配列を決定する

　最後に、参照配列と検出変異を入れ替えて、実習サンプルの配列を決定しましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/consensus_making.png?raw=true" height="250px">



### 6a. 変異入れ替えのための準備

　変異を入れ替えするためには、先ほど作った vcf.gz形式ファイルのインデックスファイルを作る必要があります。そのインデックスファイルは `bcftools index` で作成できます。

```sh
## bcftools index の使い方
bcftools index SAMPLE.vcf.gz
```

In [ ]:
%%bash
bcftools index

### 6b. 変異の入れ替えによる実習サンプルの配列決定
　参照配列と検出変異の入れ替えを `bcftools consensus` でおこないましょう。

```sh
## bcftools consensus の使い方
bcftools consensus -f REF.fasta SAMPLE.vcf.gz > SAMPLE.fasta

## オプション等の説明
# -f: 参照配列
# >: 出力先ファイル。FASTA形式で出力されるため「.fasta」の拡張子を付けるのがよい
```

In [ ]:
%%bash
bcftools consensus

 `cat` コマンドで、出来上がった SAMPLE.fasta の中身を確認できます。

In [ ]:
%%bash
cat

## 前半まとめ

　ここまでで、ひとつの実習サンプルのシーケンシングリードに対する処理をおこなってきました。同様の操作を各サンプルのデータに対しておこなうことで、それぞれの配列を決定できます。
- シーケンシングリードの情報確認
- リードのフィルタリング
- 参照配列へのリードアライメント
- 変異検出
- 変異入れ替えによる実習サンプルの配列決定

　解析実習後半では、各サンプルの配列データを使って、遺伝子系統樹の作成をおこない、種間・種内の関係を調べていきます。

<small>※ 解析実習の後半パートで使用する各サンプルの配列データは、あらかじめこちらで準備しています。</small>

\>> [後半パートのテキストへ](https://colab.research.google.com/github/CropEvol/lecture/blob/master/FFBCexpr_2023spr/FFBC2023spr_CropEvol_p2.ipynb)